# 02: Basic Transformations in PySpark

This notebook covers the following:
|#|Basics|
|--|---|
|1|Dropping NaN values|
|2|Filling missing values|
|3|Imputing values|
|4||
|5||


### Imports

In [ ]:
# Imports
import pyspark
import numpy as np
import pandas as pd
import os
from pyspark.sql import SparkSession

# Creating the spark session
spark = SparkSession.builder.appName("Practice").getOrCreate()

### Loading Dataframe and Creating Session

In [80]:
# This needs a new import. The datatypes you'll use are places after 'import'.
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Example schema definition (you need to adjust this to your actual CSV columns)
schema = StructType([
    StructField("index", IntegerType(), True),
    StructField("airline", StringType(), True),
    StructField("flight", StringType(), True),
    StructField("source_city", StringType(), True),
    StructField("departure_time", StringType(), True),
    StructField("stops", StringType(), True),
    StructField("arrival_time", StringType(), True),
    StructField("destination_city", StringType(), True),
    StructField("class", StringType(), True),
    StructField("duration", DoubleType(), True),
    StructField("days_left", IntegerType(), True),  
    StructField("price", IntegerType(), True),   
])

# Load with predefined schema
df = spark.read.option("header", "true").schema(schema).csv("./datasets/airlines_flights_data.csv")

# Printing Schema
df.printSchema()

# If any start showing as null, it's probably because you skipped a column.

root
 |-- index: integer (nullable = true)
 |-- airline: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- source_city: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- stops: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- destination_city: string (nullable = true)
 |-- class: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- days_left: integer (nullable = true)
 |-- price: integer (nullable = true)



---
## Handling Missing Data

In [ ]:
from pyspark.sql.functions import when, lit

# Don't worry about this too much, it just makes a dataframe 
cols = df.columns
df = df.select(
    [when(df["index"] == 1, lit(None)).otherwise(df[c]).alias(c) for c in cols]
)

df.show(2)

+-----+--------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops|arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|       Night|          Mumbai|Economy|    2.17|        1| 5953|
| NULL|    NULL|   NULL|       NULL|          NULL| NULL|        NULL|            NULL|   NULL|    NULL|     NULL| NULL|
+-----+--------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+
only showing top 2 rows


#### Dropping all missing rows

In [82]:
# Dropping missing rows
df.na.drop().show(2)

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
only showing top 2 rows


In [83]:
# Dropping Nulls --  drop a row if it contains any nulls.
df.na.drop(how='any').show(2)

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
only showing top 2 rows


In [84]:
# Dropping Nulls -- drop a row if it contains a given amount of nulls.
df.na.drop(how='any',thresh=2).show(2)

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
only showing top 2 rows


In [85]:
# Dropping Nulls -- drop a row if all values are nulls.
df.na.drop(how='all').show(2)


+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
only showing top 2 rows


In [86]:
# Dropping Nulls -- dropping rows in a specific column.
df.na.drop(how="any", subset=["source_city"]).show(2)


+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
only showing top 2 rows


#### Filling Missing values

In [87]:
# Filling all na values with the string 'Missing'
df.na.fill('MISSING').show(2)

+-----+--------+-------+-----------+--------------+-------+------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|  stops|arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-------+------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening|   zero|       Night|          Mumbai|Economy|    2.17|        1| 5953|
| NULL| MISSING|MISSING|    MISSING|       MISSING|MISSING|     MISSING|         MISSING|MISSING|    NULL|     NULL| NULL|
+-----+--------+-------+-----------+--------------+-------+------------+----------------+-------+--------+---------+-----+
only showing top 2 rows


In [88]:
# Filling missing values -- Selecting specific columns.
df.na.fill("Missing", ['flight', 'stops']).show(2)

+-----+--------+-------+-----------+--------------+-------+------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|  stops|arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-------+------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening|   zero|       Night|          Mumbai|Economy|    2.17|        1| 5953|
| NULL|    NULL|Missing|       NULL|          NULL|Missing|        NULL|            NULL|   NULL|    NULL|     NULL| NULL|
+-----+--------+-------+-----------+--------------+-------+------------+----------------+-------+--------+---------+-----+
only showing top 2 rows


#### Imputing missing values

In [ ]:
# Imputing values that are missing.
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['duration'],
    outputCols=["{}_imputed".format(c) for c in ['duration']]
).setStrategy("mean")

# Applying the imputer
imputer.fit(df).transform(df).show()

# setStrategy("Median") also works.

+-----+---------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+------------------+
|index|  airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|  duration_imputed|
+-----+---------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+------------------+
|    0| SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|              2.17|
| NULL|     NULL|   NULL|       NULL|          NULL| NULL|         NULL|            NULL|   NULL|    NULL|     NULL| NULL|12.221053766091938|
|    2|  AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|              2.17|
|    3|  Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|              2.25|
|    4